In [13]:
import numpy as np
import pandas as pd
from bokeh.plotting import figure, show, output_notebook

## player: AI

In [14]:
class Player:

    def __init__(self, heap):
        self.history = {}
        self.distribution = np.ones((heap + 1, 3), dtype=int)
        self.cutoff = 1000

    def __call__(self, heap):
        # randomize move based on previous games
        dist = self.distribution[heap].cumsum()
        rnd = np.random.randint(dist[2])
        move = 1 if rnd < dist[0] else 2 if rnd < dist[1] else 3
        
        # store move in history
        self.history[heap] = min(heap, move)
        
        return self.history[heap]

    def learn(self, winner):
        # update move distribution
        for heap, move in self.history.items():
            if winner is self:
                self.distribution[heap][move - 1] += 1
            else:
                self.distribution[heap][move - 1] -= 1
                self.distribution[heap] += 1

        # normalize distribution to speed learning up
        normalize = np.argwhere(self.distribution.sum(axis=1) > self.cutoff)
        for heap in normalize:
            self.distribution[heap] -= self.distribution[heap].min() - 1

        # reset game history
        self.history = {}
    
    def strategy(self):
        distribution = self.distribution[1:]
        return distribution.T / distribution.sum(axis=1)

## opponents

In [15]:
def expert_opponent(heap):
    return heap % 4 or min(heap, np.random.randint(1, 4))

In [16]:
def random_opponent(heap):
    return min(heap, np.random.randint(1, 4))

In [17]:
def take_n_opponent(take):
    return lambda heap: min(heap, take)

## training

In [18]:
def play(heap, player, opponent):
    players = player, opponent
    wins = 0

    for game in range(100001):
        # update plot periodically
        if game % 10000 == 0:
            print(game, 'games, W/L ratio', wins / 10000)
            wins = 0

        # a single game
        h = heap
        while h:
            h -= players[0](h)
            players = players[1], players[0]

        winner = players[1]
        wins += winner is player
            
        # let player learn
        player.learn(winner)
        
    # plot distribution
    plot_strategy(heap, player)

In [19]:
def plot_strategy(heap, player):
    output_notebook()

    # data
    take_1, take_2, take_3 = player.strategy()
    take_2 += take_1
    take_3 += take_2
    kwargs = {'x': range(1, heap + 1), 'width': .8}

    # plot
    plot = figure(plot_width=600, plot_height=400)
    plot.vbar(**kwargs, bottom=0, top=take_1, legend='take 1', color='#a44444')
    plot.vbar(**kwargs, bottom=take_1, top=take_2, legend='take 2', color='#88a888')
    plot.vbar(**kwargs, bottom=take_2, top=take_3, legend='take 3', color='#ccccac')
    show(plot)

## learning

In [20]:
HEAP = 21

In [21]:
play(HEAP, Player(HEAP), expert_opponent)

0 games, W/L ratio 0.0
10000 games, W/L ratio 0.0093
20000 games, W/L ratio 0.014
30000 games, W/L ratio 0.0181
40000 games, W/L ratio 0.0434
50000 games, W/L ratio 0.0657
60000 games, W/L ratio 0.3191
70000 games, W/L ratio 0.4979
80000 games, W/L ratio 0.4988
90000 games, W/L ratio 0.4994
100000 games, W/L ratio 0.4994


Loading BokehJS ...

In [22]:
play(HEAP, Player(HEAP), random_opponent)

0 games, W/L ratio 0.0
10000 games, W/L ratio 0.8755
20000 games, W/L ratio 0.9547
30000 games, W/L ratio 0.964
40000 games, W/L ratio 0.9648
50000 games, W/L ratio 0.9678
60000 games, W/L ratio 0.9705
70000 games, W/L ratio 0.9701
80000 games, W/L ratio 0.9666
90000 games, W/L ratio 0.97
100000 games, W/L ratio 0.9708


Loading BokehJS ...

In [23]:
play(HEAP, Player(HEAP), take_n_opponent(1))

0 games, W/L ratio 0.0
10000 games, W/L ratio 0.997
20000 games, W/L ratio 0.9997
30000 games, W/L ratio 0.9998
40000 games, W/L ratio 1.0
50000 games, W/L ratio 0.9999
60000 games, W/L ratio 1.0
70000 games, W/L ratio 0.9999
80000 games, W/L ratio 1.0
90000 games, W/L ratio 0.9999
100000 games, W/L ratio 1.0


Loading BokehJS ...

In [24]:
play(HEAP, Player(HEAP), take_n_opponent(3))

0 games, W/L ratio 0.0
10000 games, W/L ratio 0.9712
20000 games, W/L ratio 0.9968
30000 games, W/L ratio 0.9988
40000 games, W/L ratio 0.9983
50000 games, W/L ratio 0.9996
60000 games, W/L ratio 1.0
70000 games, W/L ratio 0.9997
80000 games, W/L ratio 1.0
90000 games, W/L ratio 0.9998
100000 games, W/L ratio 1.0


Loading BokehJS ...